# Traditional ETL workflows

- This ETL workflow generates synthetic sales and product data in-memory, performs transformations through cleaning and aggregation operations, and then loads the aggregated results into an SQLite database,

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

In [ ]:
"""Generates  DataFrame for 'sales.csv'.Dont Alter this code"""
def generate_sales_data():
    data = {
        'ORDERNUM': [10187, 10121, 10134, 10145, 10159, 10168, 10180, 10188, 10201, 10211],
        'QUANTITY': [30, 34, 41, 45, 49, 36, 29, 48, 22, 41],
        'PRICEEACH': [95.7, 81.35, 94.74, 83.26, 100, 96.66, 86.13, 100, 98.57, 100],
        'SALES': [2871, 2765.9, 3884.34, 3746.7, 5205.27, 3479.76, 2497.77, 5512.32, 2168.54, 4708.44],
        'STATUS': ['Shipped']*10
    }
    df = pd.DataFrame(data)
    df['ORDERDATE'] = pd.date_range(start='2003-02-24', periods=10, freq='M')
    return df

"""Generates DataFrame for 'products.csv'.Dont Alter this code"""
def generate_products_data():
    data = {
        'product_id': [101, 102, 103, 104, 105, 106, 107, 108, 109, 110],
        'category': ['Electronics', 'Clothing', 'Kitchenware', 'Electronics', 'Clothing', 'Kitchenware', 'Electronics', 'Clothing', 'Kitchenware', 'Electronics']
    }
    return pd.DataFrame(data)

# Generate data
sales_df = generate_sales_data()
products_df = generate_products_data()

In [ ]:
print("Sales DataFrame:")
print(sales_df)
print("\nProducts DataFrame:")
print(products_df)

Sales DataFrame:
   ORDERNUM  QUANTITY  PRICEEACH    SALES   STATUS  ORDERDATE  total_amount
0     10187        30      95.70  2871.00  Shipped 2003-02-28       2871.00
1     10121        34      81.35  2765.90  Shipped 2003-03-31       2765.90
2     10134        41      94.74  3884.34  Shipped 2003-04-30       3884.34
3     10145        45      83.26  3746.70  Shipped 2003-05-31       3746.70
4     10159        49     100.00  5205.27  Shipped 2003-06-30       4900.00
5     10168        36      96.66  3479.76  Shipped 2003-07-31       3479.76
6     10180        29      86.13  2497.77  Shipped 2003-08-31       2497.77
7     10188        48     100.00  5512.32  Shipped 2003-09-30       4800.00
8     10201        22      98.57  2168.54  Shipped 2003-10-31       2168.54
9     10211        41     100.00  4708.44  Shipped 2003-11-30       4100.00

Products DataFrame:
   product_id     category
0         101  Electronics
1         102     Clothing
2         103  Kitchenware
3         104  Ele

In [ ]:
# Data transformation functions
def clean_sales(df):
    df['total_amount'] = df['QUANTITY'] * df['PRICEEACH']
    return df

sales_cleaned = clean_sales(sales_df)
sales_with_products = pd.merge(sales_cleaned, products_df, left_on='ORDERNUM', right_on='product_id')

# Additional transformations
sales_summary = sales_with_products.groupby(['ORDERDATE', 'category']).agg({
    'total_amount': 'sum',
    'QUANTITY': 'sum'
}).reset_index()

# Load: Write to a database using SQLite
engine = create_engine('sqlite:///:memory:')  # Using in-memory SQLite database
sales_summary.to_sql('sales_summary', engine, if_exists='replace', index=False)

print("ETL process completed successfully.")

ETL process completed successfully.


# ELT and the modern data stack

- demonstrates an ELT workflow where raw data is first loaded into BigQuery, and then transformations are performed using SQL within the data warehouse itself.

In [ ]:
# Install necessary packages
!pip install pandasql

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pandasql import sqldf

In [ ]:
"""Generates DataFrame for 'sales.csv' and 'products.csv'.Dont Alter this code"""
# Define the number of entries
num_entries = 100

# Generate random sales data
np.random.seed(0)
order_ids = np.arange(10100, 10100 + num_entries)
quantities = np.random.randint(20, 50, size=num_entries)
price_each = np.random.uniform(80, 100, size=num_entries).round(2)
order_dates = [datetime.now() - timedelta(days=i) for i in range(num_entries)]
sales = (quantities * price_each).round(2)
statuses = np.random.choice(['Shipped', 'Pending', 'Cancelled'], size=num_entries)

# Create DataFrame
sales_df = pd.DataFrame({
    'ORDER_ID': order_ids,
    'PRODUCT_ID': np.random.choice(order_ids, size=num_entries),  # Add PRODUCT_ID column
    'QUANTITY': quantities,
    'PRICE_EACH': price_each,
    'ORDER_DATE': order_dates,
    'SALES': sales,
    'STATUS': statuses
})

# Display the DataFrame
print("Sales DataFrame:")
print(sales_df.head())


product_ids = np.random.choice(order_ids, size=num_entries // 2, replace=False)
categories = np.random.choice(['Electronics', 'Clothing', 'Furniture'], size=num_entries // 2)

# Create DataFrame
products_df = pd.DataFrame({
    'PRODUCT_ID': product_ids,
    'CATEGORY': categories
})


print("\nProducts DataFrame:")
print(products_df.head())

Sales DataFrame:
   ORDER_ID  PRODUCT_ID  QUANTITY  PRICE_EACH                 ORDER_DATE  \
0     10100       10171        32       99.18 2024-08-08 06:15:59.790025   
1     10101       10187        35       93.06 2024-08-07 06:15:59.790033   
2     10102       10113        41       92.70 2024-08-06 06:15:59.790037   
3     10103       10158        20       99.91 2024-08-05 06:15:59.790038   
4     10104       10181        23       91.64 2024-08-04 06:15:59.790040   

     SALES     STATUS  
0  3173.76    Pending  
1  3257.10    Pending  
2  3800.70  Cancelled  
3  1998.20    Shipped  
4  2107.72    Pending  

Products DataFrame:
   PRODUCT_ID     CATEGORY
0       10177     Clothing
1       10122    Furniture
2       10161     Clothing
3       10167     Clothing
4       10113  Electronics


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set file paths
sales_file_path = '/content/drive/My Drive/sales_data.csv'
products_file_path = '/content/drive/My Drive/products_data.csv'
output_file_path = '/content/drive/My Drive/sales_summary.csv'

# Save DataFrames as CSV
sales_df.to_csv(sales_file_path, index=False)
products_df.to_csv(products_file_path, index=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Helper function to use pandasql
def pysqldf(q):
    return sqldf(q, globals())

# Transformation query
transformed_query = """
SELECT
    DATE(s.ORDER_DATE) AS sale_date,
    p.CATEGORY AS category,
    SUM(s.QUANTITY * s.PRICE_EACH) AS total_amount,
    SUM(s.QUANTITY) AS total_quantity
FROM
    sales_df s
JOIN
    products_df p ON s.PRODUCT_ID = p.PRODUCT_ID
GROUP BY
    DATE(s.ORDER_DATE), p.CATEGORY
"""

# Run the query and store the result in a DataFrame
result_df = pysqldf(transformed_query)

# Display the resulting DataFrame
print("\nTransformed Data:")
print(result_df)

# Save the transformed data to CSV
result_df.to_csv(output_file_path, index=False)

print("ELT process completed successfully.")


Transformed Data:
     sale_date     category  total_amount  total_quantity
0   2024-05-01  Electronics       1938.00              24
1   2024-05-03     Clothing       3872.80              40
2   2024-05-05    Furniture       3917.25              45
3   2024-05-08     Clothing       2087.02              23
4   2024-05-11     Clothing       3851.65              47
5   2024-05-12     Clothing       3147.54              38
6   2024-05-14    Furniture       2679.02              31
7   2024-05-15     Clothing       4273.34              43
8   2024-05-17    Furniture       2093.92              23
9   2024-05-20     Clothing       2215.62              27
10  2024-05-21     Clothing       2048.97              21
11  2024-05-22     Clothing       1994.58              21
12  2024-05-24    Furniture       3944.96              46
13  2024-05-27  Electronics       2148.00              24
14  2024-05-28     Clothing       3233.30              35
15  2024-05-30  Electronics       3384.40            

# Choosing between ETL and ELT

- This example provides a simple framework for evaluating the factors that influence the choice between ETL
and ELT. In practice, the decision may involve more nuanced considerations and might benefit from a hybrid
approach in some cases

In [ ]:
def choose_integration_approach(data_volume, transformation_complexity, real_time_requirements, data_sensitivity, target_system_capabilities):
    score_etl = 0
    score_elt = 0

    if data_volume == 'high':
        score_etl += 1
    else:
        score_elt += 1

    if transformation_complexity == 'high':
        score_etl += 1
    else:
        score_elt += 1

    if real_time_requirements:
        score_etl += 1
    else:
        score_elt += 1

    if data_sensitivity == 'high':
        score_etl += 1
    else:
        score_elt += 1

    if target_system_capabilities == 'high':
        score_elt += 1
    else:
        score_etl += 1

    if score_etl > score_elt:
        return 'ETL'
    elif score_elt > score_etl:
        return 'ELT'
    else:
        return 'Consider hybrid approach or further analysis'

# Example usage of the function
approach = choose_integration_approach(
    data_volume='high',
    transformation_complexity='low',
    real_time_requirements=True,
    data_sensitivity='low',
    target_system_capabilities='high'
)

print(f"Recommended approach: {approach}")

Recommended approach: ELT
